## Avaliação Continuada (AC2) - Perceptron

Nesse notebook, iremos ver como aplicar o método Perceptron na nossa base de dados que foi processada no notebook `train_data.ipynb`.

Para começar então, vamos importar o notebook `train_data.ipynb` que irá conter tudo que precisamos.

In [1]:
import import_ipynb
import train_data

importing Jupyter notebook from train_data.ipynb


,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


,Category,Message
count,5572,5572
unique,2,5157
top,ham,"Sorry, I'll call later"
freq,4825,30


Shape antes de remover os dados.
(5572, 2)

Shape após remover os dados.
(5157, 2)

Quantidade de itens removidos: 415
Um dos textos antes do processamento:


'I sent you  &lt;#&gt;  bucks'

O mesmo texto, só que agora processado:


'sent you bucks'

Algumas palavras que iremos usar para montar o nosso vocabulário:
['go', 'until', 'jurong', 'point', 'crazy', 'available', 'only', 'in', 'bugis', 'great', 'world', 'la', 'buffet', 'cine', 'there', 'got', 'amore', 'wat']
Tamanho do Vocabulario:  3632
[ 1.59629866e-01 -1.10554872e-02 -2.85293967e-01 -2.42690906e-01
  7.21771181e-01  7.32151717e-02  5.67456186e-02 -1.87724203e-01
 -1.22484542e-01  2.24399030e-01  7.39724413e-02  2.45748281e-01
  3.30503613e-01  6.04098774e-02 -3.94032076e-02 -1.07717223e-01
  4.71422911e-01 -5.60647011e-01 -2.26922616e-01  4.46216166e-01
 -3.27260315e-01 -1.47730947e-01 -6.72238693e-02  9.02423859e-02
  1.76793739e-01  2.30422914e-01 -5.84167168e-02  6.83692545e-02
 -3.92336249e-01 -8.17217231e-02 -5.36404587e-02 -3.05875409e-02
  5.70999980e-01  1.69829965e-01 -5.70866108e-01  6.26165047e-02
  2.05268517e-01 -9.98219922e-02 -2.20045820e-02  3.49807084e-01
 -1.45130694e-01 -2.09874123e-01 -2.20444903e-01 -8.18783939e-02
 -4.05738167e-02 -5.75792976e-02 -6

## Treinamento

Agora, com os dados separados, vamos começar o treinamento usando o método Perceptron.

Primeiro, importamos o método que iremos usar.

In [2]:
import sklearn
from sklearn.linear_model import Perceptron

Depois, podemos definir algumas variáveis para customizar o comportamento do nosso modelo.

In [3]:
n_interations = 100 # vai controlar a quantidade de interações que será realizada no modelo, também chamado de epochs
learning_rate = 0.1 # a constante que representa a taxa de aprendizado

Depois, criamos o nosso modelo da seguinte forma:

In [4]:
perceptron = Perceptron(max_iter=n_interations, eta0=learning_rate, random_state=0)

E ai, já podemos começar o treinamento do modelo.

In [5]:
perceptron.fit(train_data.x_train, train_data.y_train) 

Perceptron(eta0=0.1, max_iter=100)

E com o modelo treinado, podemos começar a ver as métricas de acurácia do nosso modelo tentando predizer os valores de teste que separamos na etapa anterior.

In [6]:
y_pred = perceptron.predict(train_data.x_test) 

Com os valores que foram classificados acima, podemos ver qual foi a sua performance usando o `sklearn` da seguinte forma:

In [7]:
results = sklearn.metrics.classification_report(train_data.y_test, y_pred, target_names=['ham', 'spam'])

print(results)

              precision    recall  f1-score   support

         ham       0.93      0.98      0.95      1352
        spam       0.77      0.49      0.60       196

    accuracy                           0.92      1548
   macro avg       0.85      0.73      0.78      1548
weighted avg       0.91      0.92      0.91      1548



Com a tabela acima, podemos tirar algumas conclusões como: 

A precisão para identificar mensagens verdadeiras é maior do que a precisão do que identificar mensagens de spam. E isso não necessariamente é ruim porque é melhor errar mais mensagens de spam do que mensagens verdadeiras.

Dessa forma, no pior dos casos ainda vemos o spam e ignoramos mas com a mensagem real indo parar no spam, é mais dificil de ir lá olhar.

E além disso, podemos checar a nossa acurácia com mais precisão tanto dos valores de teste quanto dos valores de treino da seguinte forma:

In [8]:
accuracyTest = train_data.accuracy_score(train_data.y_test, perceptron.predict(train_data.x_test))
accuracyTrain = train_data.accuracy_score(train_data.y_train, perceptron.predict(train_data.x_train))

print("Acurácia sobre os valores de teste: ", accuracyTest)
print("Acurácia sobre os valores de treino: ", accuracyTrain)

Acurácia sobre os valores de teste:  0.9166666666666666
Acurácia sobre os valores de treino:  0.9340537545026323


E aqui vemos, temos, assim como mostrado no relatório anterior, certa de 93% e 93% de acurácia sobre os valores de teste e de treino, respectivamente.

## Cross-Validation

Apesar de eu não entender tão bem como funciona a validação cruzada, eu quis trazer nesses métodos para ter um pequena noção de como a ordem dos dados de treinamento influenciam o nosso modelo.

Dessa forma, o cross-validation permite que a gente separe em partições os nossos dados, e eles por consequencia, serão usados para treinar várias vezes o modelo e assim podemos obter as métrica de acurácia de cada vez que foi treinado o modelo.

Com essas métricas, podemos verificar se a mudança na ordem dos dados tem uma mudança significativa na acurácia do nosso modelo.

Vamos começar então com a importação das bibliotecas que iremos usar.

In [9]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit

Depois, vamos criar algumas variáveis para configurarmos como será a cross-validation.

In [10]:
n_splits = 5 # o número de partições
test_size = 0.3 # a porcentagem que será usada para teste, o restante é usada no treino
cv = ShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=0) # define a função que irá realizar o rearranjo dos dados de treino e como será divido os dados.

E assim, podemos agora chamar a função `cross_val_score` que irá nos retornar a acurácia, assim como, o desvio da acurácia que nos diz quanto a ordem os dados está influenciando o modelo.

In [11]:
scores = cross_val_score(perceptron, train_data.x, train_data.y, cv=cv)

print("%0.2f acurácia com o desvio de %0.2f" % (scores.mean(), scores.std()))

0.91 acurácia com o desvio de 0.02


É interessante ver que a acurácia praticamente se manteve na mesma, em 91%.

Isso foi um comportamento estranho comparado a outras execuções, eu executei isso em dois dias diferentes e tive resultados diferentes, apesar de setar e tentar deixar os mesmos estados randomicos (random_state=0), ele ainda teve uma diferença ao ser executado em dias diferentes.

E professor, se você chegou até aqui, seria interessante ver mais sobre cross-validation nas aulas.